In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from FPN import Features, FPNetwork , classificationhead , bboxhead
from Loss import Lossfunction
from dataset_convert import AnchorGenerator, FaceDetectionDataset
device = torch.device("mps")

In [15]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model = model.features.to(device)

Using cache found in /Users/vipulagarwal/.cache/torch/hub/pytorch_vision_v0.10.0


In [8]:
train_data = torch.load("/Users/vipulagarwal/Documents/AI & ML/Projects/New_FR/version2/faces2.0/path/to/data/widerface/dataset640.pt")
val_data = torch.load("/Users/vipulagarwal/Documents/AI & ML/Projects/New_FR/version2/faces2.0/path/to/data/widerface/val_data640.pt")


/var/folders/pc/3dvbqf1x7ts37h_0ccg5yh5r0000gn/T/ipykernel_11214/1332796123.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load("/Users/vipulagarwal/

In [16]:
extractor = Features(model,['3','6', '13','18'])
topdown = FPNetwork(out_channels=256)
classifier = classificationhead(channels=256, num_anchors= 12, num_of_classes= 1)
bboxregression = bboxhead(channels= 256 , num_anchors= 12)
loss =Lossfunction()
anchors = AnchorGenerator()
data = FaceDetectionDataset(train_data,anchors)
vali_data = FaceDetectionDataset(val_data,anchors)

In [10]:
# hyperparametrs
batch_size = 16
epochs = 1
learning_rate = 1e-3

In [11]:
training_data = DataLoader(data, batch_size=batch_size, shuffle = True)
validation_data = DataLoader(vali_data, batch_size=batch_size, shuffle = True)

In [12]:

def forward(p):
    features = extractor.extract(p)
    newfeatures = topdown(features)
    output = {}
    for key in list(newfeatures.keys()):
        temp = {}
        temp["bbox"] = bboxregression(newfeatures[key])
        temp["cls"] = classifier(newfeatures[key])
        output[key] = temp
        
    return output

def train(epochs:int,training_data , validation_data=None):
    extractor.train()
    topdown.train()
    classifier.train()
    bboxregression.train()
    optimizer = optim.Adam(list(extractor.parameters())+
                           list(topdown.parameters())+
                           list(classifier.parameters())+
                           list(bboxregression.parameters()), lr=learning_rate)
    loSS = {}
    for i in range (0,epochs):
        train_loss= 0
        for key, (image,target) in enumerate(training_data):
            epoch_loss = {}
            model_pred = forward(image)
            optimizer.zero_grad()
            ll = loss(model_pred, target)
            train_loss+=ll
            if (key % 10 == 0):
                epoch_loss[key] = train_loss/10
                print(f"The avg loss for {key}th data is {train_loss/10}")
                train_loss = 0    
            ll.backward()
            optimizer.step()
        loSS[i] = epoch_loss
    return loSS

In [20]:
total_params=sum(p.numel() for p in extractor.parameters())
total_params+= sum(p.numel() for p in topdown.parameters())
total_params+= sum(p.numel() for p in classifier.parameters())
total_params+= sum(p.numel() for p in bboxregression.parameters())

print(total_params)

7452536


In [25]:
data[0]

(tensor([[[0.4863, 0.4863, 0.4863,  ..., 0.5843, 0.7961, 0.9843],
          [0.1569, 0.1569, 0.1569,  ..., 0.3020, 0.6745, 0.9804],
          [0.1451, 0.1451, 0.1451,  ..., 0.3020, 0.6706, 0.9882],
          ...,
          [0.2510, 0.2667, 0.2745,  ..., 0.2275, 0.6392, 1.0000],
          [0.2471, 0.2510, 0.2471,  ..., 0.2510, 0.6471, 0.9961],
          [0.2784, 0.2627, 0.2392,  ..., 0.2510, 0.6471, 0.9961]],
 
         [[0.5373, 0.5373, 0.5373,  ..., 0.5961, 0.8039, 0.9882],
          [0.2118, 0.2118, 0.2118,  ..., 0.3098, 0.6745, 0.9765],
          [0.2039, 0.2039, 0.2039,  ..., 0.2980, 0.6667, 0.9804],
          ...,
          [0.2784, 0.2941, 0.3020,  ..., 0.2314, 0.6392, 1.0000],
          [0.2745, 0.2745, 0.2745,  ..., 0.2549, 0.6471, 0.9922],
          [0.3020, 0.2863, 0.2627,  ..., 0.2549, 0.6471, 0.9922]],
 
         [[0.6314, 0.6314, 0.6314,  ..., 0.6353, 0.8078, 0.9529],
          [0.4039, 0.4039, 0.4039,  ..., 0.4039, 0.7137, 0.9804],
          [0.4667, 0.4667, 0.4667,  ...,

In [24]:
a =5 
int.__add__(a,6)

11